In [1]:
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add
 
import os
import matplotlib.pyplot as plt
import numpy as np
import math
 

Using TensorFlow backend.
/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/myenv/lib/python3.7/site-pa

In [2]:
from keras.models import Sequential
from keras.applications import ResNet50

from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans

import numpy as np 
import os
import cv2
import matplotlib.pyplot as plt

import glob

In [3]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input

In [4]:
resnet_weights_path = '/Users/reejungkim/Documents/Git/YOLO/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

In [5]:
def extract_vector(path):
    resnet_feature_list = []

    for im in os.listdir(path) : #glob.glob(path):
        if im.endswith('png'):
            im = cv2.imread(path+'/' +im)
            #plt.imshow(im)
            im = cv2.resize(im,(224,224))
            img = preprocess_input(np.expand_dims(im.copy(), axis=0))
            resnet_feature = my_new_model.predict(img)
            resnet_feature_np = np.array(resnet_feature)
            resnet_feature_list.append(resnet_feature_np.flatten())

    return np.array(resnet_feature_list)

In [6]:
folder_path =''

In [7]:
array = extract_vector(folder_path)

In [9]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(array)
print(kmeans.labels_)

[5 6 6 6 6 3 7 3 3 8 5 1 3 7 5 8 5 5 1 9 8 1 5 7 2 8 3 4 4 7 4 3 2 9 0 7 1
 1 6 9 7 6 1 7 5 8 5 4 4 4 1 4 2 4 1 9 4 2 8 3 9 7 9 5 9 7 6 5 3 9 1 4 7 1
 1 3 4 6 8 8 9 5 1 5 7 8 5 2 6 6 5 2 6 9 4 0 1 0 2 4 5 4 0 9 0 6 6 6 7 3 7
 1 7 0 7 5 2 6 6 4 5 8 4 2 0 9 2 5 4 5 5 7 0 6 5 7 0 3 4 5 4 5 2 5 5 2 0 4
 5 4 6 1 4 9 6 4 1 0 9 2 5 6 6 0 1 0 1 1 4 9 3 7 1 4 5 9 5 6 9 2 2 7 0 7 1
 8 6 7 1 3 1 3 3 8 7 0 5 3 2 8 5 8 8 9 7 1 1 2 5 5 6 2 9 6 9 6 1 7 6 9 2 0
 7 1 7 0 6 7 5 9 9 0 9 8 2 6 5 6 0 3 5 0 4 0 9 1 1 9 9 1 2 7 0 9 5 9 9 7 4
 7 0 8 7 5 3 0 3 5 3 4 5 4 6 2 7 5 4 5 7 9 1 5 1 5 0 1 6 4 2 7 3 5 0 1 6 1
 9 0 7 5 6 4 6 7 4 4 9 6 4 7 1 7 9 2 4 5 6 2 8 6 8 3 3 8 3 4 7 1 5 9 5 5 9
 9 4 8 0 5 0 3 5 5 9 6 3 9 6 7 4 9 6 7 1 5 6 3 9 9 4 3 5 1 6 4 2 5 9 9 4 9
 1 5 1 1 9 2 0 4 7 5 6 1 3 5 3 2 4 3 5 5 5 1 9 4 7 5 7 5 7 4 9 4 1 2 0 7 9
 3 2 4 0 0 4 3 3 2 9 9 4 0 3 9 0 4 9 9 9 2 6 4 2 9 9 6 1 7 3 9 9 4 1 3 9 6
 2 3 6 5 9 0 3 5 5 6 1 4 4 7 8 5 1 6 9 9 0 6 0 4 8 0 4 5 6 9 0 5 1 9 0 9 4
 1 5 3 9 5 1 6 7 4 0 7 6 

In [11]:
clusters = kmeans.fit_predict(array)

In [12]:
clusters

array([5, 6, 6, 6, 6, 3, 7, 3, 3, 8, 5, 1, 3, 7, 5, 8, 5, 5, 1, 9, 8, 1,
       5, 7, 2, 8, 3, 4, 4, 7, 4, 3, 2, 9, 0, 7, 1, 1, 6, 9, 7, 6, 1, 7,
       5, 8, 5, 4, 4, 4, 1, 4, 2, 4, 1, 9, 4, 2, 8, 3, 9, 7, 9, 5, 9, 7,
       6, 5, 3, 9, 1, 4, 7, 1, 1, 3, 4, 6, 8, 8, 9, 5, 1, 5, 7, 8, 5, 2,
       6, 6, 5, 2, 6, 9, 4, 0, 1, 0, 2, 4, 5, 4, 0, 9, 0, 6, 6, 6, 7, 3,
       7, 1, 7, 0, 7, 5, 2, 6, 6, 4, 5, 8, 4, 2, 0, 9, 2, 5, 4, 5, 5, 7,
       0, 6, 5, 7, 0, 3, 4, 5, 4, 5, 2, 5, 5, 2, 0, 4, 5, 4, 6, 1, 4, 9,
       6, 4, 1, 0, 9, 2, 5, 6, 6, 0, 1, 0, 1, 1, 4, 9, 3, 7, 1, 4, 5, 9,
       5, 6, 9, 2, 2, 7, 0, 7, 1, 8, 6, 7, 1, 3, 1, 3, 3, 8, 7, 0, 5, 3,
       2, 8, 5, 8, 8, 9, 7, 1, 1, 2, 5, 5, 6, 2, 9, 6, 9, 6, 1, 7, 6, 9,
       2, 0, 7, 1, 7, 0, 6, 7, 5, 9, 9, 0, 9, 8, 2, 6, 5, 6, 0, 3, 5, 0,
       4, 0, 9, 1, 1, 9, 9, 1, 2, 7, 0, 9, 5, 9, 9, 7, 4, 7, 0, 8, 7, 5,
       3, 0, 3, 5, 3, 4, 5, 4, 6, 2, 7, 5, 4, 5, 7, 9, 1, 5, 1, 5, 0, 1,
       6, 4, 2, 7, 3, 5, 0, 1, 6, 1, 9, 0, 7, 5, 6,